In [14]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.memory import MemorySaver
import os
from typing import Dict, List, Tuple
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage


In [15]:
# Establecer la API key desde variable de entorno
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_KEY")

# 1. Create a memory saver for conversation history
memory = MemorySaver()

# 2. Create chat model
model = ChatOpenAI(temperature=0.7)

# 3. Define the prompt template
prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente conversacional amigable y útil."),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [16]:
# 4. Define state
def get_state():
    return {"history": []}

# 5. Define the conversation step
def conversation_step(state, input):
    # Get chat history from state
    history = state["history"]
    
    # Format input with history for the model
    messages = prompt.format_messages(history=history, input=input)
    
    # Get response from model
    response = model.invoke(messages)
    
    # Update history with new messages
    new_history = history + [HumanMessage(content=input), response]
    
    # Return updated state
    return {"history": new_history, "response": response.content}

In [21]:
# Initialize state
state = get_state()

# Process first user input
result = conversation_step(state, "Hola, Mi nombre es Kike")
state = {"history": result["history"]}  # Update state
print(result["response"])

# Process follow-up question
result = conversation_step(state, "¿Qué día es hoy?")
# result = conversation_step(state, "¿Quien descibrio el continente americano?")
state = {"history": result["history"]}  # Update state
print(result["response"])

# Process follow-up question
result = conversation_step(state, "¿Recuerdas como me llamo?")
state = {"history": result["history"]}  # Update state
print(result["response"])

¡Hola Kike! ¿En qué puedo ayudarte hoy?
Hoy es [fecha]. ¿Hay algo más con lo que pueda ayudarte?
¡Sí, claro! Tu nombre es Kike. ¿Hay algo más en lo que pueda ayudarte, Kike?
